In [1]:
# Created by Nigel Noll
# nigel.noll@gmail.com; nnoll@tulane.edu


import pandas as pd
#from datetime import datetime
#from dateutil.parser import parse
#from datetime import timedelta
import math
#import statistics

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 2000)


#import pandas_profiling
#df.profile_report()


In [2]:
# load  files
# one file is the complete download for 2024 directly from Insights
# The other file is a list of the fields I am using/want to keep

df = pd.read_csv('2024 complete dataset download.csv', usecols = ['Name', 'Value', 'Metric description', 'School ID', 'IPEDS ID'])
df.rename(columns = {'Value': 'performance', 'Metric description': 'metric', 'Name': 'school'}, inplace= True)
df_fields = pd.read_csv('fields to use.csv')
df_fields = df_fields[~df_fields.use.isna()]

In [3]:
# remove fields I don't use from the insights file
# in the file to identify fields I use, I marked metric fields with Y and rank fields with R
# I ended up not really using this in the final code below

df = pd.merge(left= df,  right = df_fields, on = 'metric', how = 'inner')
df.rename(columns = {'use': 'value_type'}, inplace = True)
df['value_type'] = df.value_type.map(lambda x:  'score' if x == 'Y' else  'rank')
df['performance'] = df.performance.astype('float64')

In [4]:
# I don't think I use this really. I had included this just in case when I first wrote the code

df_ranks = df[df.value_type == 'rank']
df_performance = df[df.value_type == 'score']

In [5]:
# I don't know that this is necessary, but I did this anyways. This is something I always do when working with student data

df = df[~df.duplicated()]

In [6]:
# I pivot the long version of the insights file to put each metric in its own column

df = df.pivot( index=['school', 'School ID', 'IPEDS ID'], columns='metric', values='performance').reset_index()

#.to_csv('wide metrics and ranks.csv', index = False)
df = df[~df.isna()['Overall Rank National Universities']]

In [7]:
# I created a new file with faculty salaries adjusted for priec parity. 
#I am replacing the salary in the original file with the adjusted salary value

df_salaries = pd.read_csv('adjusted faculty salaries.csv', usecols = [ 'school', 'Adjusted Salaries'])
df_salaries.rename(columns = {'Adjusted Salaries': 'Average Faculty Compensation (CY)'}, inplace=True)
df.drop(columns = ['Average Faculty Compensation (CY)'], inplace=True)
df = pd.merge(left = df, right = df_salaries, on = 'school')

# I made school my index
df.index = df['school']

In [8]:
# this function handles missing values when performing calculations below



def fill_no_rank(rank, score, method='highest'):
    #method is used to identify instances where a lower score is better such as debt and s/f ratio

    #check if missing value for metric rank - this was for testing purposes
    for x in list(df[df[rank].isna()]['school']):
        missing_a_rank.append(x)
        df.loc[x, [score]] = df[score].min()
    
    #impute missing values with closest ranked school
    if method == 'highest':
        for school in list(df[df[score].isna()]['school']):
            # if higher scores exist, use min of schools ranked better
            if df[df[rank] <= df[df['school'] == school][rank].max()][score].count() > 0:
                df.loc[school, [score]] = df[df[rank] <= df[df['school'] == school][rank].max()][score].min()
            else:
                df.loc[school, [score]] = df[score].max() 
                
    elif method == 'lowest':
        for school in list(df[df[score].isna()]['school']):
            # if lower scores exist, use max of schools ranked better
            if df[df[rank] <= df[df['school'] == school][rank].max()][score].count() > 0:
                df.loc[school, [score]] = df[df[rank] <= df[df['school'] == school][rank].max()][score].max()
            else:
                df.loc[school, [score]] = df[score].min() 
        
    return


        

In [9]:
# values I am going to use later

USE_FOR_CALC = [
    'metric_retention',
    'metric_grad_rate',
    'Average 6-year graduation rate', #use in grad perform calcs
    'Predicted graduation rate', #use in grad perform calcs
    'metric_grad_rate_performance',
    'pell_grad_rate_score', # not sure if I need this
    'metric_pell_grad_rate_percentile',
    'pell_grad_perform_score',
    'metric_pell_perform_percentile',
    'metric_f_gen_rate_percentile',
    'f_gen_grad_rate_score',
    'f_gen_grad_rate_perform_score',
    'metric_f_gen_rate_perform_percentile',
    'metric_debt',
    'metric_earnings',
    'metric_peer_score',
    'metric_sf_ratio',
    'metric_faculty_salary',
    'metric_ft_faculty',
    'metric_spend_per_student_percentile',
    'metric_pubs_top_25',
    'metric_pubs_top_5',
    'metric_cite_per_pub',
    'metric_cite_impact',
    'Educational expenditures per student'
       
]

In [10]:
#calculate grad/retention metrics

fill_no_rank('Graduation rate rank*', 'Average 6-year graduation rate')
fill_no_rank('First-year retention rate rank*', 'Average first year student retention rate')



df['metric_grad_rate'] = df['Average 6-year graduation rate']
df['metric_retention'] = df['Average first year student retention rate']
df['metric_grad_rate_performance'] = df['Average 6-year graduation rate'] / df['Predicted graduation rate']

In [11]:
# Pell Student Calculations

#pell grad rate
df['pell_percent_adjusted'] = df['Pell Students Proportion (fall 2016 entering class)'].map(lambda x: 50 if x >50 else x)
df['pell_grad_rate_score'] = (df[~df['Pell Graduation Rate'].isna()]['Pell Graduation Rate'] 
                              + df[~df['Pell Graduation Rate'].isna()]['pell_percent_adjusted'])
fill_no_rank('Pell grant graduation rate rank*', 'pell_grad_rate_score')
df['metric_pell_grad_rate_percentile'] = df['pell_grad_rate_score'].rank(method='min', ascending = True) / df['pell_grad_rate_score'].count()

#pell perform grad rate
df['pell_grad_perform_quotient'] = (df[~df['Pell Graduation Rate'].isna()]['Pell Graduation Rate'] 
                             / df[~df['Pell Graduation Rate'].isna()]['Non-Pell gradrate'])
df['pell_grad_perform_quotient'] = df['pell_grad_perform_quotient'].map(lambda x: 1 if x > 1 else x) 
df['pell_grad_perform_score'] = (df[~df['Pell Graduation Rate'].isna()]['pell_grad_perform_quotient'] 
                          + df[~df['Pell Graduation Rate'].isna()]['pell_percent_adjusted'] * .01)
fill_no_rank('Pell grant comparative graduation rate rank*', 'pell_grad_perform_score')

df['metric_pell_perform_percentile'] = df['pell_grad_perform_score'].rank(method='min', ascending = True) / df['pell_grad_perform_score'].count()

In [12]:
# First Gen Calculations

#f-gen grad rate
df['f_gen_grad_rate_score'] = ((df['First generation graduation rate (latest year)'] + df['First generation student % (latest year)'] 
      + df['First generation graduation rate (one year older)'] + df['First generation student % (one year older)']
      + df['First generation graduation rate (two years older)'] + df['First generation student % (two years older)'])) / 3
fill_no_rank('First generation graduation rate rank*', 'f_gen_grad_rate_score')
df['metric_f_gen_rate_percentile'] = df['f_gen_grad_rate_score'].rank(method='min', ascending = True) / df['f_gen_grad_rate_score'].count()


#f-gen grad rate perform
df['f_gen_grad_rate_perform_quotient'] = (df['First generation graduation rate'] / df['Non-first-generation gradrate']) 
df['f_gen_grad_rate_perform_score'] = (df['f_gen_grad_rate_perform_quotient'].map(lambda x: 100 if x > 100 else x) 
                                    + df['First generation proportion'] * .01)
fill_no_rank('First generation graduation rate performance rank*', 'f_gen_grad_rate_perform_score')
df['metric_f_gen_rate_perform_percentile'] = df['f_gen_grad_rate_perform_score'].rank(method='min', ascending = True) / df['f_gen_grad_rate_perform_score'].count()

In [13]:
# grad debt

df['metric_debt'] = (df['Median debt (latest year)'] + df['Median debt (older year)']) / 2
fill_no_rank('Borrower debt rank*', 'metric_debt', 'lowest')

#grad earning
df['metric_earnings'] = df['College grads earning more than a HS grad (%)'].map(lambda x: 90 if x > 90 else x) - 90
fill_no_rank('College grads earning more than a HS grad rank*', 'metric_earnings')

In [14]:
# peer score / sf ratio / salary / ft faculty / expend

df['metric_peer_score'] = df['Peer assessment score']
fill_no_rank('Peer Assessment Rank*', 'metric_peer_score')

# sf ratio
df['metric_sf_ratio'] = df['Student/faculty ratio']
fill_no_rank('Student to faculty ratio rank*', 'metric_sf_ratio', 'lowest')

#faculty salary
df['metric_faculty_salary'] = df['Average Faculty Compensation (CY)']
fill_no_rank('Faculty salary rank*', 'metric_faculty_salary')

#ft faculty
df['metric_ft_faculty'] = df['% of faculty who are full-time']
fill_no_rank('% of faculty who are full-time rank*', 'metric_ft_faculty')

# expend
fill_no_rank('Financial resources rank*', 'Educational expenditures per student')
df['metric_spend_per_student_percentile'] = df['Educational expenditures per student'].rank(method='min', ascending = True) / df['Educational expenditures per student'].count()


In [15]:
# publication data
# this data comes from a messy set of calculations I ran in a PBI file. 
# I am not sharing this approach. It was a challenge because the <5,000 publication correction
# I do not have a recommendation on a good way to handle this and do not want to share my questionable approach
# The approach I used was based on a recommendation from my President's office, and the approach was just fill in anything that was close enough
# Honestly, I have no idea how they did the < 5,000 pub adjustment
# My team laughs at the absurd comments I write in my files
# I actually like Bob Morse. I hear he's truly a good guy and not the evil person some people portray him as
# However, when Bob Morse retires, I think Bob Morse should take up a career reading bed time stories.

df_pubs = pd.read_csv('pub data for calc.csv')

pub_col_names  = {'* PUBLICATION TOP 25 FOR CALC': 'metric_pubs_top_25',
 '* PUBLICATION TOP 5 FOR CALC': 'metric_pubs_top_5',
 '*CITE PER PUB CORRECTION FOR CALC': 'metric_cite_per_pub',
 '* CITE IMPACT CORRECTION FOR CALC': 'metric_cite_impact'}

df_pubs.rename(columns = pub_col_names, inplace=True)
df_pubs.index = df_pubs['school']
df_pubs.drop(columns = 'school', inplace=True)
df.drop(columns='school', inplace=True)
df = pd.merge(left= df, right = df_pubs, on = 'school', how='left')
df['school'] = df.index

fill_no_rank('Publications Cited in Top 25% of Journals Rank+*', 'metric_pubs_top_25')
fill_no_rank('Publications Cited in Top 5% of Journals Rank+*', 'metric_pubs_top_5')
fill_no_rank('Citations per publication Rank+*', 'metric_cite_per_pub')
fill_no_rank('Field Weighted Citation Impact Rank+*', 'metric_cite_impact')


<h1> have all but test scores </h1>

In [16]:
# calculate means, std, and z-scores

metrics =[    'metric_retention',
    'metric_grad_rate',
    'metric_grad_rate_performance',
    'metric_pell_grad_rate_percentile',
    'metric_pell_perform_percentile',
    'metric_f_gen_rate_percentile',
    'metric_f_gen_rate_perform_percentile',
    'metric_debt',
    'metric_earnings',
    'metric_peer_score',
    'metric_sf_ratio',
    'metric_faculty_salary',
    'metric_ft_faculty',
    'metric_spend_per_student_percentile',
    'metric_pubs_top_25',
    'metric_pubs_top_5',
    'metric_cite_per_pub',
    'metric_cite_impact']

metrics.append('school')
df_calc = df[metrics]
metrics.remove('school')
    
    
for metric in metrics:
    new_col_name = metric + '_z'
    metric_mean = df_calc[metric].mean()
    metric_std = df_calc[metric].std()
    df_calc[new_col_name] = (df_calc[metric] - metric_mean) / metric_std
    

#fix sf ratio
df_calc['metric_sf_ratio_z'] = df_calc['metric_sf_ratio_z'] * -1
#fix debt
df_calc['metric_debt_z'] = df_calc['metric_debt_z'] * -1    
    
for metric in metrics:
    mean_col = metric + '_mean'
    df_calc[mean_col] = df_calc[metric].mean()

for metric in metrics:
    std_col = metric + '_std'
    df_calc[std_col] = df_calc[metric].std()
  

C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\909707593.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc[new_col_name] = (df_calc[metric] - metric_mean) / metric_std
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\909707593.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc[new_col_name] = (df_calc[metric] - metric_mean) / metric_std
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\909707593.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [17]:
# add in weighted z-scores
# NOTE GRAD RATE CALC ADJUSTED BECAUSE NO TEST SCORES CURRENTLY IN MODEL

metric_weights = { 'metric_retention': .05,
    'metric_grad_rate': .21, #BASED ON NO TEST SCORES
    'metric_grad_rate_performance': .1,
    'metric_pell_grad_rate_percentile': .03,
    'metric_pell_perform_percentile': .03,
    'metric_f_gen_rate_percentile': .025,
    'metric_f_gen_rate_perform_percentile': .025,
    'metric_debt': .05,
    'metric_earnings': .05,
    'metric_peer_score': .2,
    'metric_sf_ratio': .03,
    'metric_faculty_salary': .06,
    'metric_ft_faculty': .02,
    'metric_spend_per_student_percentile': .08,
    'metric_pubs_top_25': .005,
    'metric_pubs_top_5': .01,
    'metric_cite_per_pub': .0125,
    'metric_cite_impact': .0125}

for metric in metrics:
    new_col_name = metric + '_weighted'
    df_calc[new_col_name] = df_calc[metric + '_z'] * metric_weights[metric]

C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\4091020799.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc[new_col_name] = df_calc[metric + '_z'] * metric_weights[metric]
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\4091020799.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc[new_col_name] = df_calc[metric + '_z'] * metric_weights[metric]
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\4091020799.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [18]:
# I renamed the columns

col_names = list(df_calc.columns)
col_names.remove('school')

for col in col_names:
    df_calc.rename(columns={col: col[7:]}, inplace = True)




C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\2676200497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc.rename(columns={col: col[7:]}, inplace = True)
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\2676200497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_calc.rename(columns={col: col[7:]}, inplace = True)
C:\Users\NollNigel\AppData\Local\Temp\ipykernel_2612\2676200497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [19]:
#used to look at overall values across all schools

df_calc.to_csv('calculator.csv', index=False)

In [20]:
#used to calculate values for Tulane

df_tulane = df_calc[df_calc.school == 'Tulane University']
df_tulane.melt().to_csv('tulane calc.csv', index=False)

In [21]:
#pull other values for calc
#these are used to calculate percentiles in calculator (contains a few extra fields that are not needed)

additional_values = []

for col in USE_FOR_CALC:
    if col.find('metric') < 0:
        additional_values.append(col)   
        
df[additional_values].to_csv('additional calc values.csv')

<h1> THATS RIGHT </h1>

In [22]:
df_tulane

,retention,grad_rate,grad_rate_performance,pell_grad_rate_percentile,pell_perform_percentile,f_gen_rate_percentile,f_gen_rate_perform_percentile,debt,earnings,peer_score,sf_ratio,faculty_salary,ft_faculty,spend_per_student_percentile,pubs_top_25,pubs_top_5,cite_per_pub,cite_impact,school,retention_z,grad_rate_z,grad_rate_performance_z,pell_grad_rate_percentile_z,pell_perform_percentile_z,f_gen_rate_percentile_z,f_gen_rate_perform_percentile_z,debt_z,earnings_z,peer_score_z,sf_ratio_z,faculty_salary_z,ft_faculty_z,spend_per_student_percentile_z,pubs_top_25_z,pubs_top_5_z,cite_per_pub_z,cite_impact_z,retention_mean,grad_rate_mean,grad_rate_performance_mean,pell_grad_rate_percentile_mean,pell_perform_percentile_mean,f_gen_rate_percentile_mean,f_gen_rate_perform_percentile_mean,debt_mean,earnings_mean,peer_score_mean,sf_ratio_mean,faculty_salary_mean,ft_faculty_mean,spend_per_student_percentile_mean,pubs_top_25_mean,pubs_top_5_mean,cite_per_pub_mean,cite_impact_mean,retention_std,grad_rate_std,grad_rate_performance_std,pell_grad_rate_percentile_std,pell_perform_percentile_std,f_gen_rate_percentile_std,f_gen_rate_perform_percentile_std,debt_std,earnings_std,peer_score_std,sf_ratio_std,faculty_salary_std,ft_faculty_std,spend_per_student_percentile_std,pubs_top_25_std,pubs_top_5_std,cite_per_pub_std,cite_impact_std,retention_weighted,grad_rate_weighted,grad_rate_performance_weighted,pell_grad_rate_percentile_weighted,pell_perform_percentile_weighted,f_gen_rate_percentile_weighted,f_gen_rate_perform_percentile_weighted,debt_weighted,earnings_weighted,peer_score_weighted,sf_ratio_weighted,faculty_salary_weighted,ft_faculty_weighted,spend_per_student_percentile_weighted,pubs_top_25_weighted,pubs_top_5_weighted,cite_per_pub_weighted,cite_impact_weighted
school,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Tulane University,93.0,86.25,0.9375,0.542529,0.14023,0.011494,0.002299,20875.0,-11.0,3.6,7.8,120974.0,85.7,0.933333,62,18,0.58,1.64,Tulane University,1.232725,1.297675,-0.402493,0.18424,-1.248986,-1.684178,-1.725823,0.19809,-0.210216,1.189157,1.561934,0.850973,0.415842,1.495411,0.793998,0.696065,1.023276,1.078478,80.99387,65.246552,0.992771,0.489148,0.500864,0.497862,0.501023,21645.964368,-9.54023,2.735862,14.754023,98198.28046,79.926897,0.501149,48.951724,12.618391,0.363701,1.124253,9.739501,16.185452,0.137322,0.289735,0.288742,0.288787,0.288977,3891.999502,6.944156,0.726681,4.452189,26764.314836,13.882927,0.289007,16.433639,7.731474,0.211379,0.478218,0.061636,0.272512,-0.040249,0.005527,-0.03747,-0.042104,-0.043146,0.009904,-0.010511,0.237831,0.046858,0.051058,0.008317,0.119633,0.00397,0.006961,0.012791,0.013481


In [23]:
# I am pulling a file that has everything. This is not needed for anything 

df.to_csv('all calc data from python.csv')